# PyTorch MNIST example dissected

In this notebook we'll explore the components of the
[PyTorch MNIST example](https://github.com/pytorch/examples/tree/master/mnist)
one-by-one.

* Part 1: [Loading the data](1_mnist_load.ipynb)
* Part 2: [Model components and forward propagation](2_mnist_model.ipynb)
* Part 3: [Autodiff and backpropagation](3_mnist_backprop.ipynb) <-- **you are here**
* Part 4: [Training the model](4_mnist_train.ipynb)
* Part 5: [Visualizing the results](5_mnist_visualize.ipynb)

## 3 Backpropagation

Before we start training our model, let's explore the auto-differentiation functionality of PyTorch.

In fact, PyTorch has some good [online documentation](https://pytorch.org/docs/stable/notes/autograd.html) on the subject; Below we will focus more on the autograd internals.

In [1]:
import torch

Recall in [Part 2](2_mnist_model.ipynb) parameters of the neural net had a flag `require_grad=True`. It indicates that this tensor will participate in backpropagation:

In [2]:
i = torch.eye(2, requires_grad=True)
x = torch.tensor([1., 2.], requires_grad=False)

y = i.matmul(x)

print("I * X =", y, "requires_grad =", y.requires_grad)

I * X = tensor([1., 2.], grad_fn=<MvBackward>) requires_grad = True


If at least one operand has `required_grad` flag set, the result will also have `requires_grad=True`.

Note that the result tensor also has the `grad_fn` property - we've seen that in [Part 2](2_mnist_model.ipynb), too. It looks like a function or a callable object. Let's take a closer look.

In [3]:
help(y.grad_fn)

Help on MvBackward object:

class MvBackward(object)
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  register_hook(...)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  metadata
 |  
 |  next_functions
 |  
 |  requires_grad



So it is a callable indeed. It allso has the `requires_grad` flag, and what looks like a link to the next element (or elements?) in backpropagation chain. Let's feed some data trough the `.__call__()` method.

In [4]:
z = torch.tensor([3., 4.])

y.grad_fn(z)

(tensor([[3., 6.],
         [4., 8.]]), None)

Since `y` is the result of the matrix-vector multiplication of matrix `i` and vector `x`, the gradient should propagate to both operands. However, vector `x` has `requires_grad=False`, and therefore we do not propagate data to it. Hence the second element of the result tuple is `None`.

In [5]:
(grad_i, grad_x) = y.grad_fn(z)

z.reshape((2,1)).matmul(x.reshape((1,2)))

tensor([[3., 6.],
        [4., 8.]])

In [21]:
x = torch.tensor(2., requires_grad=True)
y = torch.tensor(3., requires_grad=True)
z = x * 4
z

tensor(8., grad_fn=<MulBackward>)

In [22]:
z.grad_fn(torch.tensor(0.1))

tensor(0.4000)